In [ ]:
import pandas as pd
import numpy as np
import os
import heapq

In [ ]:
data_users = pd.read_excel("Data.xlsx", sheet_name="Users")
data_tracking = pd.read_excel("Data.xlsx", sheet_name="Tracking")

In [ ]:
data_users.head()

In [ ]:
data_tracking = data_tracking.drop(labels=[' accessType;;;;;', 'Unnamed: 9',
       'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13',
       'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17',
       'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21',
       'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25',
       'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29',
       'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33'], axis=1)
data_tracking.columns = ['idUser', 'day', 'month', 'year', 'idContent', 'contentSubType',
       'contentBranch', 'contentName']

In [ ]:
data_tracking.head()

In [ ]:
# Query example
# user_test = data_tracking[data_tracking.idUser == 17208]

In [ ]:
# v1.0 Get all the viewed content for each user
user_dict = {"user": "list"}
user_list = []

for i in range(0, len(data_tracking["idUser"])):

    # Save & Create new user_list if this row starts a new users data.
    if i == 0 or data_tracking["idUser"].iloc[i] != data_tracking["idUser"].iloc[i-1]:
        user_dict[data_tracking["idUser"].iloc[i-1]] = user_list
        user_list = []
        
    user_list.append(data_tracking["idContent"].iloc[i])
    

In [ ]:
user_dict[1880842]

In [ ]:
# Get similarity v1.0

user_dict_keys = user_dict.keys()
similarity_dict = {"user": "other_user"}

# Loop over all users in pairs.
for i in range(0, 1):
    inner_similarity_dict = {"other_user": "similarity"}
    
    for j in range(0, 15):
        similarity = 0
        # Check all of both users item lists.
        for k in range(0, len(user_dict[user_dict_keys[i]])):
            for l in range(0, len(user_dict[user_dict_keys[j]])):
                if user_dict[user_dict_keys[i]][k] == user_dict[user_dict_keys[i]][m]:
                    similarity = similarity + 1
        
        inner_similarity_dict[user_dict[user_dict_keys[i]]] = similarity
        
    similarity_dict[user_dict_keys[i]] = inner_similarity_dict

In [ ]:
# v2.0 Get all the viewed content for each user

user_data = pd.DataFrame(columns=["idUser", "idContent", "timesViewed"])
data_dict = {"idUser": "user", "idContent": "content", "timesViewed": "viewed"}
user = []
content = []
viewed = []

for i in range(0, len(data_tracking["idUser"])):
    if i == 150*10**3:
        break

    # Save & Create new user_list if this row starts a new users data.
    if i != 0 and data_tracking["idUser"].iloc[i] != data_tracking["idUser"].iloc[i-1]:
        data_dict = {"idUser": user, "idContent": content, "timesViewed": viewed}
        this_user_data = pd.DataFrame.from_dict(data_dict)
        user_data = pd.concat([user_data, this_user_data])
        user = []
        content = []
        viewed = []
    
    item_found = False

    for j in range(len(content), 0, -1):
        if content[j-1] == data_tracking["idContent"].iloc[i]:
            viewed[j-1] += 1  
            item_found = True
            break
        
    if len(content) == 0 or item_found == False:
        user.append(data_tracking["idUser"].iloc[i]) 
        content.append(data_tracking["idContent"].iloc[i]) 
        viewed.append(1)
        
    # save the user data frame to save time
    user_data.to_csv("df_user_data.csv")

In [ ]:
# load user data from csv.
user_data = pd.read_csv("df_user_data.csv")
user_data = user_data.drop("Unnamed: 0", axis=1)

In [ ]:
user_data.tail(15)

In [ ]:
#v1.0 find similarity for a specific user

def find_similarity(userID):
    this_user_data = user_data[user_data["idUser"] == userID]
    simililarity_dict = {}

    for i in range(0, len(user_data["idUser"])):
        if user_data["idUser"].iloc[i] != userID:                        
            if user_data["idUser"].iloc[i] in simililarity_dict:
                simililarity_dict.update( {user_data["idUser"].iloc[i] : round(simililarity_dict[user_data["idUser"].iloc[i]] + 1/len(this_user_data["idContent"]), 4)} )
            else:
                simililarity_dict[user_data["idUser"].iloc[i]] = round(1/len(this_user_data["idContent"]), 4)
            
    return simililarity_dict

In [ ]:
user_list = []
for index, key in enumerate(user_data["idUser"]):
    if index == 25:
        break
    elif user_data["idUser"].iloc[index] == user_data["idUser"].iloc[index-1]:
        pass
    else:
        print(key)
        user_list.append(find_similarity(key))
    
    

In [ ]:
user_list[0]

In [ ]:
top_5 = heapq.nlargest(5, user_list[0], key=user_list[0].get)
top_5